# Notes on Nucleo STM32

Using STM32CubeIDE

Help for the board is in `C:\Users\j05625pe\STM32Cube\Repository\STM32Cube_FW_F0_V1.11.2\Drivers\STM32F0xx_HAL_Driver`

Tutorial: https://www.youtube.com/watch?v=oI-CQBKWbDY&list=PLnMKNibPkDnGtuIl5v0CvC81Am7SKpj02&index=37

## MX perspective

1. Select board. My Nucleo is STM32F091RC
2. Choose pins functionality and configure
    a. If there is overlap, CTRL click to get alternatives.
3. Choose clocks.
4. Generate code.

## C/C++ perspective

1. Modify main function. Use HAL functions for using core system and peripherals.
2. Build.
3. Debug or run.

## Interrupts

1. In MX use GPIO pins as EXTI.
2. Enable in NVIC and set priorities (HAL_Delay works with SYSCLK which needs higher priority (lower value) than EXTI to work in a Callback).
3. Look for EXTI handler in stm32f0xx_it.c. That function uses a function in stm32f0xx_hal_gpio.c that is associated to a Callback function with \__weak definition.
4. Copy that function (without \__weak) to the USER CODE 0 section in main.c, and define in the USER CODE 4 section.
5. Use a switch GPIO_Pin case when different sources of EXTI are possible, to know which one produced the interrupt.

## UART

1. Configure using MX
2. HAL_UART_Transmit (Receive) (Blocking functions)
3. Check COM for STLink in Device Manager (COM4)
4. RealTerm to watch port.

### With Interrupt (_IT)

Callbacks for every sent/received byte?

1. Recover Callback functions from driver src and take to main to define.
2. Before main while, HAL_UART_Receive_IT or Transmit only once.
3. Identify which USART is working in the Callback with a switch case.
4. The HAL_UART_Receive_IT has to be called in the callback to keep waiting the following RX.

### With DMA stream

Callbacks only when the transfer is complete.

1. Do not enable NVIC for the UART.
2. Add the rx and tx DMA and generate.
3. Use USART callbacks for doing something when the DMAs are full.

## Timers

1. Configure Timer (Prescaler -1, period -1...)
2. Initialized timers are not running timers. Use HAL_TIM_Base_Start_IT()
3. Look for desired Callback in hal_tim.c -> HAL_TIM_PeriodElapsedCallback()

In 1, define the source as internal clock or external trigger to use timer as counter.

### PWM

1. Configure Pin as TIM-CH.
2. Configure the PWM duty cycle as a count number below the period auto-reset.
3. Use HAL_TIM_PWM_Start.

## ADC

1. Choose one ADC input pin.
2. Configure. Select a sampling time measured in clock cycles.
3. Enable interrupt. NVIC
4. Use HAL_ADC_Start_IT in main (before while).
5. Get the Callback from drivers. HAL_ADC_ConvCpltCallback() and define outside main function.
6. Each conversion (interrupt) requires a HAL_ADC_GetValue(), result goes to uint16_t as it has 12 bits.
7. Start again with HAL_ADC_Start_IT.

### STMStudio

1. Open STMStudio
2. File-Import Variables : Debug/\*.elf -> Select and Import
3. Right Click -> send to var viewer
4. Set upper value
5. Play (Debugger off)

### DMA

For using the DMA "Continuous conversion mode" and "DMA continuous requests" have to be enabled. Then, is similar to using DMA with USART. The function expects a uint32_t* buffer but it uses the type of data from the configuration (half word). It does not require the callback because the values are stored in the buffer.

# Linear actuators

[RS 918-1331](https://uk.rs-online.com/web/p/miniature-electric-actuators-rod/9181331/) Actuonix Micro Linear Actuator - 20% Duty Cycle , 12V dc, 50mm, L12-50-210-12-P £110.02

A 5V pulse of 1ms fully retracts the actuator and 2ms fully extends. 2 M4 mounting.

1. Orange – Feedback Potentiometer negative reference rail
2. Purple – Feedback Potentiometer wiper
3. Red – Motor V+ (6V or 12V)
4. Black – Motor V- (Ground)
5. Yellow – Feedback Potentiometer positive reference rail

No built in controller. Analog position feedback signal that can be input to an external controller. While voltage is applied to the motor power pins, (3 & 4) the actuator extends. Reverse the polarity and the actuator retracts. This can be accomplished manually with a DPDT switch or relay, or using an H-Bridge circuit. Position of the actuator stroke can be monitored by providing any stable low and high reference voltage on pins 1 & 5, then reading the position signal on pin 2. The voltage on pin 2 will vary linearly between the two reference voltages in proportion to the position of the actuator stroke. 

The L12 –P actuator can be used as a linear servo by connecting the actuator to an external controller such as the LAC board offered by Firgelli. This control board reads the position signal from the L12, compares it with your input control signal then commands the actuator to move via an onboard H-bridge circuit. The LAC allows any one of  the following control inputs: Analog 0-5V or 4-20mA, or Digital 0-5V PWM, 1-2ms Standard RC, or USB. The RC input effectively transforms your L12 into a linear servo, which is a direct replacement for any common hobby servo used in RC toys and robotics. Refer to the LAC datasheet for more details. 

## Possible control boards

- [RS 134-6445](https://uk.rs-online.com/web/p/power-motor-robotics-development-tools/1346445/) Digilent H-Bridge Expansion Module 410-069 £7.70
- [Farnell 3335842](https://uk.farnell.com/stmicroelectronics/ev-vnhd7008ay/eval-baord-h-bridge-dc-motor-driver/dp/3335842?st=h%20bridge) EVAL BAORD, H-BRIDGE, DC MOTOR DRIVER £13.14
- [Farnell 3267858](https://uk.farnell.com/infineon/hbridgekit2gotobo1/eval-board-dc-motor-control/dp/3267858?st=h%20bridge) HBRIDGEKIT2GOTOBO1 - EVAL BOARD, DC MOTOR CONTROL £14.15 *No, for using with PC, USB*
- [RS 9181372](https://uk.rs-online.com/web/p/motor-controllers/9181372/?sra=pstk) Actuonix, Linear Actuator Control Board, Analogue, Digital Control, 5 → 24 V dc, 4 A, Panel Mount


# I2C Thermocouple

https://wiki.seeedstudio.com/Grove-I2C_Thermocouple_Amplifier-MCP9600/

## Connection

- [RS 797-9168](https://uk.rs-online.com/web/p/pcb-headers/7979168/) Molex, Sherlock, 35362, 4 Way, 1 Row, Straight PCB Header x10 £2.04
- [RS 512-8608](https://uk.rs-online.com/web/p/pcb-headers/5128608/) JST, PA, 4 Way, 1 Row, Straight PCB Header x10 £3.02
- [RS 820-1456](https://uk.rs-online.com/web/p/crimp-contacts/8201456/) JST, PH Female Crimp Terminal Contact 24AWG BPH-002T-P0.5S x100 £1.70

Not compatible:
- [RS 174-3233](https://uk.rs-online.com/web/p/shields-for-arduino/1743233) Seeed Studio, Grove Base Shield V2 https://www.seeedstudio.com/Base-Shield-V2.html

I2C Hub:
- [RS 174-3256](https://uk.rs-online.com/web/p/communication-wireless-development-tools/1743256/) Seeed Studio I2C Hub Grove - I2C Hub for Connecting Multiply I2c Devices £2.22 https://www.seeedstudio.com/Grove-I2C-Hub.html
- [RS 179-3726](https://uk.rs-online.com/web/p/development-tool-accessories/1793726/) Seeed Studio 110990210 for use with Grove-Base Shield
- [RS 179-3734](https://uk.rs-online.com/web/p/development-tool-accessories/1793734/) Seeed Studio 110990028 for use with 2.54mm Pitch Pin Header

In Nucleo-F091RC:
- I2C1 : SCL(PA9): 1-CN5(D8), 21-CN10, SDA(PA10): 3-CN9(D2), 33-CN10
- I2C2 : SCL(PB10): 7-CN9(D6), 25-CN10, or (PB13): *30-CN10*, SDA(PB11): *18-CN10*

## Project

I2C_Temp. Use of UART and I2C without interrupts, but a timer is used (interrupt) to ask for temperature and inform via UART.

## Standard Pinout

- VCC : 5V or 3.3V

## ADDR

VDD (0x67), GND (0x60, default) or voltage divider (8 addresses).

- 10 kohm to VDD: 
    - [RS 740-8805](https://uk.rs-online.com/web/p/surface-mount-fixed-resistors/7408805/) Bourns 10kΩ, 0603 (1608M) Thick Film SMD Resistor ±1% 0.1W - CR0603-FX-1002ELF x50 £0.25
    - [Rapid 72-9849](https://www.rapidonline.com/10k-0603-1-1-10w-royal-ohm-chip-resistor-pack-of-100-72-9849) 10K 0603 1% 1/10W Royal Ohm Chip Resistor - Pack of 100 £0.496
- (2.2, 4.3, 7.5, 13, 22,  or 43) kohm, 5% tolerance, to GND
    - [RS 199-5518](https://uk.rs-online.com/web/p/surface-mount-fixed-resistors/1995518/) Yageo 2.2kΩ, 0603 (1608M) Thick Film SMD Resistor ±1% 0.1 W, 0.2 W - RC0603FR-072K2L x100 £0.50
    - [RS 679-0469](https://uk.rs-online.com/web/p/surface-mount-fixed-resistors/6790469/) Vishay 4.3kΩ, 0603 (1608M) Thick Film SMD Resistor ±1% 0.1W - CRCW06034K30FKEA x50 £0.45
    - [RS 199-5459](https://uk.rs-online.com/web/p/surface-mount-fixed-resistors/1995459/) Yageo 13kΩ, 0603 (1608M) Thick Film SMD Resistor ±1% 0.1 W, 0.2 W - RC0603FR-0713KL x100 £0.50
    - [RS 199-5636](https://uk.rs-online.com/web/p/surface-mount-fixed-resistors/1995636/)Yageo 7.5kΩ, 0603 (1608M) Thick Film SMD Resistor ±1% 0.1 W, 0.2 W - RC0603FR-077K5L x100 £0.50
    - [Rapid 72-9846](https://www.rapidonline.com/7k5-0603-1-1-10w-royal-ohm-chip-resistor-pack-of-100-72-9846) 7K5 0603 1% 1/10W Royal Ohm Chip Resistor - Pack of 100 £0.407
    - [Rapid 72-9851](https://www.rapidonline.com/13k-0603-1-1-10w-royal-ohm-chip-resistor-pack-of-100-72-9851) 13K 0603 1% 1/10W Royal Ohm Chip Resistor - Pack of 100 £0.4.72    
    - [Rapid 72-9838](https://www.rapidonline.com/2k2-0603-1-1-10w-royal-ohm-chip-resistor-pack-of-100-72-9838) 2K2 0603 1% 1/10W Royal Ohm Chip Resistor - Pack of 100 £0.496
    - [Rapid 72-9855](https://www.rapidonline.com/22k-0603-1-1-10w-royal-ohm-chip-resistor-pack-of-100-72-9855) 22K 0603 1% 1/10W Royal Ohm Chip Resistor - Pack of 100 £0.496
    
Solder tip 0.5-0.8 mm.

### Available solder stations

**Room 2.07**

<img src="./Files/BP_53_2.07.png" width="200">

Weller PT or ET. [RS 661-655](https://uk.rs-online.com/web/p/soldering-iron-tips/0661655/) Weller PT P8 0.8 mm Straight Conical Soldering Iron Tip £3.16 *Maybe too big*

[Rapid 85-6744](https://www.rapidonline.com/xytronic-44-413081-0-5mm-pointed-soldering-tip-for-lf-3000-lf-855d-85-6744) Xytronic 44-413081 0.5mm Pointed Soldering Tip for LF-3000 & LF-855D *Too small*

**Room 1.05**

Antex 690 SD soldering station with SD50 soldering iron.

Can work with: Antex TL2EJ70 TC50 50W Soldering Iron [Rapid 85-4780](https://www.rapidonline.com/antex-tl2ej70-tc50-50w-soldering-iron-85-4780), [Rapid 85-1216](https://www.rapidonline.com/antex-b1105l0-long-life-tip-1105l-0-5mm-85-1216) Antex B1105L0 Long Life Tip 1105L 0.5mm £3.44

**Room B.02 (1270)**

Solomon SL-20. Same tips as BP-53.

# Relays

[Seeed Studio](https://wiki.seeedstudio.com/Relay_Shield_v3/), Relay Shield V3.0 [RS 174-3234](https://uk.rs-online.com/web/p/shields-for-arduino/1743234/?sra=pstk) 68.7x53.5x30.8 £15.25

Interface via Arduino digital I/O pins 4,5,6, and 7 (PB5, PB4, PB10, PA8).

First try in Test_00.

In [1]:
vdd = 2.87
R1 = 10e3
R2 = 22e3
print(f'Output = {vdd*R2/(R1+R2)}')

Output = 1.973125
